# BIG DATA ANALYSIS : 딥러닝을 이용한 주시가격 예측
## RNN을 이용한 주시가격 예측
---

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow


In [ ]:
#관련 라이브러리 임포트
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn import utils
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import pandas_datareader.data as web

tf.random.set_seed(777) 

## 1. 주식 데이터 로드

In [ ]:
df = web.DataReader("005300.KS", "yahoo", "2013-01-01", "2020-09-15")


## 2. EDA

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.tail(n=7)

## 3. 전처리

In [ ]:
df = df[['Open', 'High', 'Low', 'Close']]

data = df.to_numpy()
split = len(data) - int(len(data)*0.3)
print(split)
print(df.iloc[split])
train = data[:split]
test = data[split:]


#데이터를 스케일링 해서 학습 효과 향상
transformer = MinMaxScaler()
train = transformer.fit_transform(train)
test = transformer.transform(test)

sequence_length = 7
window_length = sequence_length + 1

x_train = []
y_train = []
for i in range(0, len(train) - window_length + 1):
    window = train[i:i + window_length, :]
    x_train.append(window[:-1, :])
    y_train.append(window[-1, [-1]])
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = []
y_test = []
for i in range(0, len(test) - window_length + 1):
    window = test[i:i + window_length, :]
    x_test.append(window[:-1, :])
    y_test.append(window[-1, [-1]])
x_test = np.array(x_test)
y_test = np.array(y_test)


print(x_train.shape)
print(y_train.shape)


In [ ]:
df.tail(n=10)

In [ ]:
x_test[-1]

In [ ]:
transformer.inverse_transform([[-0.16731017, -0.15589569, -0.17207148, -0.16925065]])

In [ ]:
y_train[-2]

## 4. 모델 학습 및 검증

In [ ]:
##########모델 학습
##########모델 검증

input = Input(shape=(sequence_length, 4))
net = LSTM(units=512)(input) 
net = Dense(units=512, activation='relu')(net)
net = Dense(units=1)(net)
model = Model(inputs=input, outputs=net)

model.summary()

model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.01))
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test)) 


## 5. 결과확인

In [ ]:
##########모델 예측

y_test_inverse = []
for y in y_test:
    inverse = transformer.inverse_transform([[0, 0, 0, y[0]]])
    y_inverse = inverse.flatten()[-1]
    y_test_inverse.append(y_inverse)

y_predict = model.predict(x_test)
y_predict_inverse = []
for y in y_predict:
    inverse = transformer.inverse_transform([[0, 0, 0, y[0]]])
    y_inverse = inverse.flatten()[-1]
    y_predict_inverse.append(y_inverse)

import matplotlib.pyplot as plt
plt.plot(y_test_inverse, c="r", label="true")
plt.plot(y_predict_inverse, c="b", label="pred")
plt.xlabel('Time Period')
plt.ylabel('Close')
plt.legend()
plt.show()


## 7. 다음 주의 주가는 과연!!

In [ ]:
df = web.DataReader("005300.KS", "yahoo", "2020-09-16")
df

In [ ]:
df = df[['Open', 'High', 'Low', 'Close']]


x_test = df.to_numpy()

x_test = transformer.transform(x_test)
x_test = np.expand_dims(x_test, 0)
print(x_test)

y_predict = model.predict(x_test)
print(y_predict) #[[0.6553757]]
print(y_predict.flatten()[0]) #0.6553757

inverse = transformer.inverse_transform([[0, 0, 0, y_predict.flatten()[0]]])
print(inverse.flatten()[-1])

In [ ]:
#Q 여러분들이 관심 갖고 계신 주식으로 한번 해볼까요?